A mixture of $N =10$ random Gaussians centered in $[-1,1]^n$ 
$$
f_{\mu,\sigma}(x, y) = \sum_{i=1}^{N} \frac{1}{\sigma_i\sqrt{2\pi}}\exp\left(-\frac{(x-\mu_i)^2}{2\sigma_i^2}\right)
$$
 is defined over the square $[-1, 1]^2$.

In [11]:
using Pkg
using Revise 
Pkg.activate("../../.")
using Globtim
using DynamicPolynomials, DataFrames
using ProgressLogging
using Optim
using GLMakie

  Activating project at `~/globtim`


In [12]:
# Constants and Parameters
const n, a, b = 2, 1, 1
const scale_factor = a / b       # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = 0.5, 1 / 10  # Sampling parameters
const tol_l2 = 1.5e-3             # Define the tolerance for the L2-norm

0.0015

In [13]:
N = 12
params = init_gaussian_params(n, N, 1.2, 0.1)
# Create a closure that captures params
rand_gaussian_closure = (x) -> rand_gaussian(x, params)
f = rand_gaussian_closure; # Function to be optimized

In [14]:
d = 24 # Initial Degree 
center = [0.0, 0.0]
TR = test_input(f,
    dim=n,
    center=center,
    GN=80, # Number of samples
    sample_range=scale_factor,
    degree_max=,
)
pol_cheb = Constructor(TR, d, basis=:chebyshev)
pol_lege = Constructor(TR, d, basis=:legendre);
@polyvar(x[1:n]) # Define polynomial ring 
real_pts_cheb = solve_polynomial_system(x, n, d, pol_cheb.coeffs; basis=:chebyshev, bigint=true)
df_cheb = process_critical_points(real_pts_cheb, f, TR)
real_pts_lege = solve_polynomial_system(x, n, d, pol_lege.coeffs; basis=:legendre, bigint=true)
df_lege = process_critical_points(real_pts_lege, f, TR)

current L2-norm: 7.221769321601344e-5
current L2-norm: 6.505234744085744e-5
Tracking 529 paths...   0%|▏                            |  ETA: 0:06:11



Tracking 529 paths...  41%|████████████                 |  ETA: 0:00:02



Tracking 529 paths...  84%|████████████████████████▎    |  ETA: 0:00:00



Tracking 529 paths... 100%|█████████████████████████████| Time: 0:00:01
  # paths tracked:                  529
  # non-singular solutions (real):  529 (17)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         529 (17)

=== Starting Critical Points Processing (dimension: 2) ===
Processed 3 points (0.0s)
Tracking 529 paths...   0%|▏                            |  ETA: 0:10:22



Tracking 529 paths...   1%|▎                            |  ETA: 0:04:31



Tracking 529 paths...   2%|▌                            |  ETA: 0:03:06



Tracking 529 paths...   2%|▌                            |  ETA: 0:02:40



Tracking 529 paths...   2%|▊                            |  ETA: 0:02:

Row,x1,x2,z
,Float64,Float64,Float64
1,0.133403,0.507382,1.1184
2,0.696959,-0.510056,3.5356
3,0.0159271,0.758902,1.26644


We need to carefully assert the type of the coefficients of the polynomial approximant. 

In [15]:
# df_cheb, df_min_cheb = analyze_critical_points(f, df_cheb, TR, tol_dist=0.05)
df_lege, df_min_lege = analyze_critical_points(f, df_lege, TR, tol_dist=0.1)

Processing point 1 of 3
Optimization has converged: ✓
Processing point 2 of 3
Optimization has converged: ✓
Processing point 3 of 3
Optimization has converged: ✓


(3×8 DataFrame
 Row │ x1         x2         z        y1          y2          close  steps     ⋯
     │ Float64    Float64    Float64  Float64     Float64     Bool   Float64   ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ 0.133403    0.507382  1.1184     0.133387    0.507319   true      3.0   ⋯
   2 │ 0.696959   -0.510056  3.5356   -44.4886     49.3175    false      1.0
   3 │ 0.0159271   0.758902  1.26644   51.7738    -23.4193    false      2.0
                                                                1 column omitted, 1×4 DataFrame
 Row │ x2        x1        value    captured 
     │ Float64   Float64   Float64  Bool     
─────┼───────────────────────────────────────
   1 │ 0.507319  0.133387   1.1184      true)

In [20]:
df_lege
pol_lege.degree

24

In [6]:
# plot_polyapprox_levelset(pol_cheb, TR, df_cheb, df_min_cheb)

In [18]:
plot_polyapprox_rotate(pol_lege, TR, df_lege, df_min_lege)

In [8]:
# plot_polyapprox_levelset(pol_lege, TR, df_lege, df_min_lege)

In [21]:
# GLMakie.closeall()